# **NoteBook Summary**

This notebook presents a complete pipeline for segmenting long videos into shorter, captioned clips based on timestamp annotations. Using OpenCV, the notebook first loads each video and extracts key properties such as frame rate (fps), resolution, and total frame count. It then converts each annotated timestamp pair (start and end times in seconds) into corresponding frame indices. To ensure that each generated clip contains a meaningful visual segment, the notebook enforces a minimum of 8 frames per clip—automatically extending the segment forward or backward if needed without exceeding the video bounds. For each segment, frames are extracted and saved into a new video file using cv2.VideoWriter. A CSV file is generated to map each clip's file path to its corresponding caption, enabling easy reference for training or evaluation in downstream tasks. Finally, the notebook includes steps to organize and upload the resulting dataset to Kaggle, including metadata creation and publishing through the Kaggle API.



A. Video Loading and Property Extraction
OpenCV Video Capture:
The code uses cv2.VideoCapture to open the original video file. The properties such as fps, total frame count, frame width, and height are obtained with cap.get() calls.
Citation: OpenCV VideoCapture Documentation

B. Frame Conversion
Timestamps to Frame Indices:
The timestamp (seconds) is multiplied by the fps to yield frame indices (e.g., start_frame = int(start_time * fps)). This conversion is standard in video processing.

C. Ensuring Minimum Number of Frames
Frame Deficit Handling:
The code checks if the number of frames in the segment is fewer than 8. If so, it first attempts to extend the segment forward. If that is not sufficient (e.g., reaching the end of the video), it then extends backward. This ensures that each clip contains at least eight frames.

D. Video Clipping and Writing
Writing Video Clips:
A loop extracts frames starting from the adjusted start_frame to end_frame and writes them into a new video file using cv2.VideoWriter.
Citation: OpenCV VideoWriter Documentation

E. CSV File Creation
Mapping Video Path and Caption:
The mapping is stored in a list and then written into a CSV file with pandas. This CSV serves as an index connecting each generated clip to its corresponding caption.

4. Useful Links and Citations
OpenCV Documentation:

Video Capture and Video Properties: OpenCV Documentation

Video Writer: OpenCV VideoWriter Documentation

Python JSON Library: Python JSON Docs

pandas: pandas Documentation

In [ ]:
import cv2
import json
import os
import math
import pandas as pd

# --- Configuration ---
json_path = "/kaggle/input/ucf-mini/normal_sampled_annotations.json"
videos_dir = "/kaggle/input/ucf-mini"
output_dir = "./splitted_clips_normal"
csv_output_path = "./splitted_clips_normal.csv"
min_frames = 8

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# --- Load JSON Dataset ---
with open(json_path, "r") as f:
    dataset = json.load(f)

csv_rows = []

# --- Process Each Video in the Dataset ---
for video_name, video_data in dataset.items():
    video_file = os.path.join(videos_dir, video_name + ".mp4")  # Adjust the extension if necessary

    # Open video capture using OpenCV
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error opening video {video_file}")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # or use 'XVID' if you prefer

    # Process each segment: iterate over timestamps and corresponding caption
    for idx, (times, caption) in enumerate(zip(video_data["timestamps"], video_data["sentences"])):
        start_time, end_time = times
        start_frame = int(start_time * fps)
        end_frame = int(end_time * fps)
        num_frames = end_frame - start_frame

        # Adjust segment if less than minimum required frames
        if num_frames < min_frames:
            deficit = min_frames - num_frames

            # Try extending forward
            if end_frame + deficit <= total_frames:
                end_frame += deficit
            else:
                # If not enough frames ahead, try extending backward
                if start_frame - deficit >= 0:
                    start_frame -= deficit
                else:
                    # Fallback: use entire range if possible
                    start_frame = 0
                    end_frame = total_frames

            # Update num_frames based on new indices
            num_frames = end_frame - start_frame

        # Set the video to the starting frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Prepare output clip file path and writer
        output_clip_path = os.path.join(f"{video_name}_clip{idx}.mp4")
        out = cv2.VideoWriter(output_clip_path, fourcc, fps, (width, height))

        # Write frames to the new clip
        frames_written = 0
        for f in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
            frames_written += 1
        out.release()

        print(f"Extracted clip: {output_clip_path} with {frames_written} frames.")

        # Append CSV mapping info: ensuring the caption is trimmed
        csv_rows.append({
            "video_path": output_clip_path,
            "caption": caption.strip()
        })

    # Reset cap to the beginning (or close and reopen for next video)
    cap.release()

# --- Save CSV Mapping ---
df = pd.DataFrame(csv_rows)
df.to_csv(csv_output_path, index=False)
print(f"CSV mapping saved to: {csv_output_path}")


In [ ]:
# --- Save CSV Mapping ---
df = pd.DataFrame(csv_rows)
df.to_csv("/kaggle/working/splitted_clips_normal/splitted_clips_normal.csv", index=False)

In [ ]:
import json
import os

# Save kaggle.json in ~/.kaggle/
with open('/kaggle/working/kaggle.json', 'w') as f:
    json.dump({"username":"nourfakih","key":"0005ac45aa3dc353c01a5d486ed0a5ac"}, f)

os.environ['KAGGLE_CONFIG_DIR'] = "/kaggle/working"

!mkdir -p ~/.kaggle
!cp /kaggle/working/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
import os
dataset_name = "splitted-clips-normal"

# os.makedirs('/kaggle/working/' + dataset_name, exist_ok=True)

# # Move the folder inside the dataset directory
# shutil.move("/kaggle/working/UCF-mini-dataset/sampled_annotations.json", "/kaggle/working/UCF-mini-dataset/{dataset_name}")

# Create the metadata file
metadata = f"""
{{
  "title": "{dataset_name}",
  "id": "nourfakih/{dataset_name}",
  "licenses": [{{"name": "CC0-1.0"}}]
}}
"""

with open("/kaggle/working/splitted_clips_normal/dataset-metadata.json", "w") as f:
    f.write(metadata)


In [ ]:
!kaggle datasets create -p /kaggle/working/splitted_clips_normal 
